## Predictive modelling

This notebook contains our attempts at modelling and predicting delays. The notebook goes chornologically through our thought processes and how the modelling went. The final model used can be found in the final_project file, this notebook focuses on the different approaches we tried.

In [ ]:
%%local 

import os
import pandas as pd
pd.set_option("display.max_columns", 50) 
pd.options.mode.chained_assignment = None
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import plotly.graph_objects as go
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)


username = os.environ['RENKU_USERNAME']
hiveaddr = os.environ['HIVE_SERVER2']
(hivehost,hiveport) = hiveaddr.split(':')
print("Operating as: {0}".format(username))

In [ ]:
%load_ext sparkmagic.magics
# If using Python 3 kernel

In [ ]:
%%local 
from IPython import get_ipython
# Set up spark session
server = 'http://iccluster029.iccluster.epfl.ch:8998'

packages = """{"packages": "graphframes:graphframes:0.6.0-spark2.3-s_2.11"}"""

# Set application name as "<your_gaspar_id>-final_project"
get_ipython().run_cell_magic(
    'spark',
    line='config', 
    cell=f"""{{ "name": "{username}-final_project", "executorMemory": "4G", "executorCores": 4, "numExecutors": 10, "driverMemory": "4G", "conf": {packages}}}"""
)
# Send username to spark channel
get_ipython().run_line_magic(
    "spark", "add -s {0}-final_project -l python -u {1} -k".format(username, server)
)

In [ ]:
%%spark
import pyspark.sql.functions as F

## Predictive model of arrival time
Here, we build a predictive model for the delay in the transport network. First, we do the following assumption: It suffices to predict the arrival time. This is because we can then calculate the departure time as:

max(plan_departure, pred_arrival + wait_time)

Where wait_time is a parameter we decide upon through finding the median in the dataset for departures that are delayed. The reason that we do this assumption is that buses and trains scarcely get delayed on the station, but rather between stations. On station, there is more of a fixed time that is needed to lett passenger go on and off the transport.

## Weather data

We use weather data to strengthen our predictive model as it probably correlates with delays.

The way we load and save the weather data below may seem strange. The reason for this is that our plan was to read it in lcoally and the use %%send_to_spark to get it into spark. However, the send_to_spark command created a seperate spark session which we only were able to access thorugh the %%pretty magic. We therefore decided to write it to HDFS and load it in again with the correct Spark session.

In [ ]:
%%local
# reading in the weather csv
weather_df = pd.read_csv('../data/weather.csv')
weather_df = weather_df.fillna(weather_df.median())
weather_df = weather_df.drop(columns='tsun')
weather_df.head()

In [ ]:
%%send_to_spark -i weather_df -t df

In [ ]:
%%pretty
# writing the file to the hdfs cluster
weather_df.write.csv('/group/gutane/weather.csv', mode='overwrite', header='True')

In [ ]:
# reading in the weather file
weather_df = spark.read.csv('/group/gutane/weather.csv', header='True')
weather_df.show(2)

## Istdaten data

In [ ]:
%%spark
# Load distinct istdaten
istdaten = spark.read.orc('/group/gutane/istdaten_distinct')

In [ ]:
# only extracting rows with stops inside the 15km radius
istdaten = istdaten.filter(istdaten.stop_id.isin(uni_stops))

## Preprocessing of the data

To make the data ready for modelling, we need to complete som preprocessing

In [ ]:
from pyspark.sql.functions import *
# removing rows where actual and prognosed arrival time does not exist
df = istdaten.filter('actual_arrival_time is not null')
df = df.filter('scheduled_arrival_time is not null')

# Calculating delay, the difference between actual and scheduled arrival time
df = df.withColumn('actual_arrival_time',to_timestamp(col('actual_arrival_time'), "dd.MM.yyyy HH:mm:ss"))\
  .withColumn('scheduled_arrival_time', to_timestamp(col('scheduled_arrival_time'), "dd.MM.yyyy HH:mm"))\
  .withColumn('delay',unix_timestamp("actual_arrival_time") - unix_timestamp('scheduled_arrival_time'))

# Calculating delay, the difference between actual and scheduled arrival time
df = df.withColumn('actual_departure_time',to_timestamp(col('actual_departure_time'), "dd.MM.yyyy HH:mm:ss"))\
  .withColumn('scheduled_departure_time', to_timestamp(col('scheduled_departure_time'), "dd.MM.yyyy HH:mm"))\

# adding weekday, month, year, and hour of the day to the table
df = df.withColumn('trip_date',to_date(col('trip_date'), "dd.MM.yyyy"))\
  .withColumn('weekday', dayofweek('trip_date'))\
  .withColumn('month', month('trip_date'))\
  .withColumn('year', year('trip_date'))\
  .withColumn('time', hour('scheduled_arrival_time'))

In [ ]:
df.printSchema()

In [ ]:
#check how common it is that delay is created at the station
count = df.filter(unix_timestamp(df.scheduled_departure_time)>
                  unix_timestamp(df.actual_departure_time)+unix_timestamp(df.scheduled_arrival_time)-unix_timestamp(df.actual_arrival_time) + 60)\
                  .count()
print("In %.4f %% of the stops, moren that one minute delay is created at the station"%(float(count)/float(total_count)*100))
print("That is in %d of %d rows"%(count, total_count))

In [ ]:
#removing rows that do not null as travel time

df = df.filter('travel_time is not null')

In [ ]:
# converting columns to correct datatype
weather_df = weather_df.withColumn('trip_date',to_date(col('date'), "dd/MM/yyyy"))\
                        .withColumn('prcp', weather_df.prcp.cast('long'))\
                        .withColumn('pres', weather_df.pres.cast('long'))\
                        .withColumn('snow', weather_df.snow.cast('long'))\
                        .withColumn('tavg', weather_df.tavg.cast('long'))\
                        .withColumn('tmax', weather_df.tmax.cast('long'))\
                        .withColumn('tmin', weather_df.tmin.cast('long'))\
                        .withColumn('wdir', weather_df.wdir.cast('long'))\
                        .withColumn('wpgt', weather_df.wpgt.cast('long'))\
                        .withColumn('wspd', weather_df.wspd.cast('long'))\

In [ ]:
# merging df and weather_df on date
df_full = df.join(weather_df, 'trip_date', 'inner')

In [ ]:
# print the new schema
df_full.printSchema()

In [ ]:
# taking a sample of the data frame for testing of code
sample = df_full.sample(0.001)
# writing the sample to hdfs of speed purposes
sample.write.orc('/group/gutane/sample', mode='overwrite')

In [ ]:
# reading the sample in from the new partitions in hdfs
sample = spark.read.orc('/group/gutane/sample')

## EDA and feature engineering

_Which features should we use? - some initial thoughts_

- PRODUCT_ID is probably smart as bus probably are more delayed than trains
- STOP_ID, some stops may be more prone to delays than others, e.g. Lausanne-Flon vs. a small village in Valais.
- Delay in departure at previous stop, this will probably be a good feature, but will be predicted by the model in real-time so I kinda doubt it will work out in practice. But worth a try.
- DAY, more delay on Monday at 08 than Sunday at 08.
- MONTH, More delay in the winter than in the summer?
- TIME_OF_DAY, More delay in rush hour than the middle of the night
- Weather, staff mentioned this as a previously succesful predictor. Makes sense that ice-->more delay, as buses slip etc. So temperature and snow for example could be good variables.
- diff scheduled departure previous stop to scheduled arrival this stop, as distance between stops should impact delay times
- cumulated trip time at this stop

In [ ]:
sample.printSchema()

In [ ]:
import datetime
# swiss holidays from 2018-2021
swiss_holidays = [datetime.date(2018, 1, 1),datetime.date(2018, 4, 1), datetime.date(2018, 3, 30),
                  datetime.date(2018, 4, 2), datetime.date(2018, 5, 10), datetime.date(2018, 5, 20),
                  datetime.date(2018, 5, 21), datetime.date(2018, 8, 1), datetime.date(2018, 12, 25),
                  datetime.date(2019, 1, 1), datetime.date(2019, 4, 21), datetime.date(2019, 4, 19),
                  datetime.date(2019, 4, 22), datetime.date(2019, 5, 30), datetime.date(2019, 6, 9),
                  datetime.date(2019, 6, 10), datetime.date(2019, 8, 1), datetime.date(2019, 12, 25),
                  datetime.date(2020, 1, 1), datetime.date(2020, 4, 12), datetime.date(2020, 4, 10),
                  datetime.date(2020, 4, 13), datetime.date(2020, 5, 21), datetime.date(2020, 5, 31),
                  datetime.date(2020, 6, 1), datetime.date(2020, 8, 1), datetime.date(2020, 12, 25),
                  datetime.date(2021, 1, 1), datetime.date(2021, 4, 4), datetime.date(2021, 4, 2),
                  datetime.date(2021, 4, 5), datetime.date(2021, 5, 13), datetime.date(2021, 5, 23),
                  datetime.date(2021, 5, 24), datetime.date(2021, 8, 1), datetime.date(2021, 12, 25)]

# adding a variable specifying whether a given day is a holiday or not
sample = sample.withColumn('holiday', sample.trip_date.isin(swiss_holidays))

In [ ]:
# calculating # unique values in the feature columns
features = ['transport_type','travel_time',
            'weekday', 'month','stop_id', 'year', 'time',
            'prcp', 'pres', 'snow', 'tavg', 'tmax', 'tmin',
            'wdir', 'wpgt', 'wspd','holiday', 'delay_departure']
label = 'delay'
unique = sample.select([F.countDistinct(col).alias(col) for col in features])

In [ ]:
%%spark -o unique -t df

In [ ]:
%%local
# inspecting the number of unique values locally
unique

In [ ]:
# calculating descriptive statistics for the dataframe
statistics = sample.describe()

In [ ]:
%%spark -o statistics -t df

In [ ]:
%%local
# inspecting descriptive statistics locally
statistics.drop(columns = ['trip_id', 'stop_id', 'transport_type', 'date'], inplace=True)
statistics.head()

In [ ]:
# as seen above, we made some mistake calculating delay_departure, namely actual-scheduled instead of scheduled-actual
# we therefore fix this here
sample = sample.withColumn('delay_departure', -sample.delay_departure)

In [ ]:
%%spark -o sample -t df 
#writing dataframe to local to plot distribution

In [ ]:
# checking if travel time is negative
print(sample.filter(sample.travel_time<0).select('travel_time').count())
print(sample.filter(sample.cumulated_travel_time<0).select('travel_time').count())

In [ ]:
# filtering out rows with negative travel time
sample = sample.filter(sample.travel_time>=0)
sample = sample.filter(sample.cumulated_travel_time>=0)

In [ ]:
%%local
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# plotting the continous features as histograms
features = ['delay', 'travel_time','cumulated_travel_time','prcp', 'pres', 'snow', 'tavg', 'tmax', 'tmin', 'wdir', 'wpgt', 'wspd', 'delay_departure']
fix, axs = plt.subplots(7,2, figsize=(24,36))
for i,element in enumerate(features):
    sns.histplot(sample[element], ax=axs[i//2, i%2])
    axs[i//2, i%2].set_title(element)

Notably, the delay distributions looks kinda log-normal or right-skewed distributed. Snow and precipation is mostly zero. 

In [ ]:
%%local
# plotting the categorical variables as barplots
features = ['transport_type', 'weekday', 'month', 'year', 'time', 'holiday']
fix, axs = plt.subplots(3,2, figsize=(24,30))
for i,element in enumerate(features):
    axs[i//2, i%2].bar(sample[element].unique(), sample.groupby(element)[element].count().values)
    axs[i//2, i%2].set_title(element)

In [ ]:
# dropping rows with na values
sample = sample.dropna()
# dropping rows where transport_type is empty
sample = sample.filter('transport_type is not null')
sample = sample.filter('transport_type != ""')

#### Doing a correlation analysis

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline
from pyspark.ml.stat import Correlation

# features to be used for correlation analysis
features = ['delay','transport_type','stop_id', 'travel_time','cumulated_travel_time',
            'weekday', 'month', 'year', 'time', 'prcp', 'pres', 'snow','tavg', 'tmax',
            'tmin', 'wdir', 'wpgt', 'wspd', 'holiday', 'delay_departure']

# selecting only relevant columns
sample_2 = sample.select(features)

# numeric indexing for the strings (indexing starts from 0)
indexer = StringIndexer(inputCol="transport_type", outputCol="t_type")
indexer2 = StringIndexer(inputCol="stop_id", outputCol="s_id")
features.remove('transport_type')
features.remove('stop_id')
# creating a vector assembler
vectorAssembler = VectorAssembler(inputCols = features + ['t_type', 's_id'], outputCol = 'features')

# applying transforms
pipeline = Pipeline(stages = [indexer, indexer2, vectorAssembler])
vector_df = pipeline.fit(sample_2).transform(sample_2)
vector_df = vector_df.select(['features', label])


In [ ]:
# creating correlation matrix
matrix = Correlation.corr(vector_df.select('features'), 'features')

In [ ]:
%%spark -o matrix -t df

In [ ]:
%%local
import numpy as np
corr_matrix = np.asarray(dict(matrix['pearson(features)']).get(0).get('values')).reshape(20,20)
corr_delay = [element[0] for element in corr_matrix]

In [ ]:
%%local
# printing correlation between delay and the explanatory variables
features = ['delay', 'travel_time','cumulated_travel_time', 'weekday', 'month', 'year', 'time', 'prcp', 'pres', 'snow', 'tavg', 'tmax', 'tmin', 'wdir', 'wpgt', 'wspd','holiday', 'delay_departure', 'transport_type', 'stop_id']
for i, element in enumerate(features):
    print("Correlation between %s and delay: %.3f"%(element, corr_delay[i]))

Generally, there is little correlation between our explanatory variables and delay. This is worrying. Delay_departure and delay is very strongly correlated, however in production, we cannot relay on accurate estimates for delay_departure for routes that have not started yet. However, it made us realize that we must use delay_departure when we can due to the high correlation. We therefore decided to create two models. One for routes in the future, and one for routes that already have started. Stop_id also have a bit correlation, but it is difficult to use as a categorical features as it can take 2k distinct values. We therefore create a new variable below, which will indicate if the stop is one with much delay or not.

In [ ]:
import numpy as np

# calculating number of rows per stop
count_stops = sample.groupBy('stop_id').count().collect()
# summing the delay per stop
sum_delay = sample.groupBy('stop_id').sum('delay').collect()

count_sum = {} # dictinaory mapping stop_id to sum_delay/count for every stop
values = [] # values of the dictionary as list
for i, element in enumerate(count_stops):
    key = element[0]
    value = sum_delay[i][1]/element[1]
    count_sum[key] = value
    values.append(value)
    
# calculating the 90th quantile of the sum_delay/count
quant = np.quantile(np.asarray(values), 0.9)

In [ ]:
# assigning 1 to stop that have top 10% delay per trip
@F.udf
def search(x):
    a = count_sum.get(x)
    if a>=quant:
        return 1
    else:
        return 0

sample = sample.withColumn('delay_prone_stop', search('stop_id').cast('double'))

In [ ]:
%%local
fig, ax = plt.subplots(1,1, figsize = (12,10))

features = ['delay', 'travel_time','cumulated_travel_time', 'weekday', 'month', 'year', 'time', 'prcp', 'pres', 'snow', 'tavg', 'tmax', 'tmin', 'wdir', 'wpgt', 'wspd','holiday', 'delay_departure', 'transport_type', 'stop_id']

sns.heatmap(corr_matrix, xticklabels = features, yticklabels = features, ax = ax)


tavg, tmax, and tmin (avg., max and min temperature) correlates strongly with each other. We therefore only keep tavg. (as keeping all won't give more info, and to avoid near Multicollinearity in the feature matrix which will lead to numerical issues in some algorithms.)

## Difference between arrival and departure times
As previously mentioned, the predictive model we build is predicting arrival time. We therefore also need a method to predict departure times based on the delay in arrival time. We here use a simple method, namely calculate the median over the transport type in the cases where the transport is delayed in arrival such that arrival is later than scheduled departure. The reason for this method was shown above when we showed that it is very scarce that delay is created at the station, but rather betwen stations.

In [ ]:
sample_new.printSchema()

In [ ]:
from pyspark.sql.functions import unix_timestamp
from pyspark.sql.functions import col

# calculating the difference of actual arrival and actual departure time
# this is how long the transport is at the station
sample_new = sample.withColumn('wait_dep', -unix_timestamp('actual_arrival_time') + unix_timestamp('actual_departure_time'))

# filtering the df on rows where actual arrival > scheduled departure
sample_new = sample_new.filter(col('actual_arrival_time') > col('scheduled_departure_time'))

# function for calculating median
magic_percentile = F.expr('percentile_approx(wait_dep, 0.5)')

# calculating median over transport_type
sample_new.groupBy('transport_type').agg(magic_percentile.alias('median_stoptime')).show()


We see our assumption is correct. When a transport vehicle is delayed, it is very quick at the stop. For bus and tram which arrived at the stop after the bus/tram was supposed to leave the stop, the median delay is only 12 seconds. For train it is 48 seconds. We therefore use these values in real time to predict departure time in the following way:

pred_departure = max(scheduled_departure, pred_arrival + median_stoptime_ttype)

## More processing

In [ ]:
# inspecting the values in categorical parts of the features
print(sample.select('transport_type').distinct().collect())
print(sample.select('weekday').distinct().collect())
print(sample.select('month').distinct().collect())
print(sample.select('year').distinct().collect())
print(sample.select('time').distinct().collect())

In [ ]:
# ensuring categorical value index for weekday and month starts with 0 to enable one hot encoder
# also making year a variable meaning num years since 2018, in theory should be like a time trend
sample = sample.withColumn('weekday', sample.weekday-1)\
                .withColumn('month', sample.month-1)\
                .withColumn('year', sample.year-2018)\

In [ ]:
# writing the processed features to orc to avoid having to repeat processing every time
sample.write.orc('/group/gutane/processed_features', mode='overwrite')

#### Finally modelling time

In [ ]:
# reading data from hdfs
sample = spark.read.orc('/group/gutane/processed_features')

Small last bit of feature engineering

In [ ]:
# creating column which is 1 if snow and 0 otherwise
sample = sample.withColumn('snow2', sample.snow>0)\

# function for interacting snow2 and transport_type to a new categorical variable
# the thesis is that ice should be more of a problem for a bus than a train
@F.udf
def interact_cat(x,y):
    if x>0:
        snow = 1
    else:
        snow = 0
    if y=='Zug':
        return snow
    elif y=='Tram':
        return 2+snow
    elif y=='Bus':
        return 4+snow
    else:
        return 6+snow

# creating the interacted column
sample = sample.withColumn('snow_ttype', interact_cat(sample.snow2,sample.transport_type).cast('int'))\

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline
from pyspark.ml.stat import Correlation

#### Not one hot encoded approach
For the random forest, we do not one hot encode categorical features

In [ ]:
# features specify which features are to be used
features = ['transport_type', 'cumulated_travel_time', 'weekday', 'month', 'time', 'snow_ttype']

# the model will aim to predict 'delay', the residual between actual and scheduled arrival time
label = 'delay'

# non one-hot encoded approach
indexer = StringIndexer(inputCol="transport_type", outputCol="t_type")
features.remove('transport_type')
# asssembling features to one sparse vector column
vectorAssembler = VectorAssembler(inputCols = features + ['t_type'], outputCol = 'features')
pipeline = Pipeline(stages = [indexer, vectorAssembler])
non_ohe_df = pipeline.fit(sample).transform(sample)
non_ohe_df = non_ohe_df.select(['features', label])

#### One hot encoding
For the rest of the models, we use one hot encoding

In [ ]:
# the features column
features = ['transport_type', 'travel_time', 'cumulated_travel_time', 'weekday', 'month', 'time', 'prcp', 'snow2', 'tavg', 'holiday', 'snow_ttype', 'delay_departure', 'delay_prone_stop']
# the model will aim to predict 'delay', the residual between actual and scheduled arrival time
label = 'delay'

# one hot encoding appraoch
indexer = StringIndexer(inputCol="transport_type", outputCol="t_type")
features.remove('transport_type')
oh_feat = ['weekday', 'month', 'time']
onehotencoders = [OneHotEncoder(inputCol = element, outputCol = 'oh_'+element) for element in oh_feat]
features.remove('weekday')
features.remove('month')
features.remove('time')
# asssembling features to one sparse vector column
vectorAssembler = VectorAssembler(inputCols = features + ['t_type','oh_weekday', 'oh_month', 'oh_time'], outputCol = 'features')
pipeline = Pipeline(stages = [indexer]+ onehotencoders + [vectorAssembler])
ohe_df = pipeline.fit(sample).transform(sample)
ohe_df = ohe_df.select(['features', label])

#### Splitting into train and test data

In [ ]:
vector_df = non_ohe_df # set to ohe_df for one hot encoded features

# splitting the data into training and test
splits = vector_df.randomSplit([0.9, 0.1])
train_df = splits[0].cache()
test_df = splits[1].cache()

#### Random forest regression
First, we implement a Random Forest Regression

In [ ]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 24 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=24).fit(vector_df)

(trainingData, testData) = (train_df, test_df)

# Train a RandomForest model.
rf = RandomForestRegressor(featuresCol="indexedFeatures", labelCol='delay', maxDepth=10)

# Chain indexer and forest in a Pipeline
pipeline = Pipeline(stages=[featureIndexer, rf])

# Train model.  This also runs the indexer.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("prediction", "delay", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="delay", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

rfModel = model.stages[1]
print(rfModel)  # summary onl

#### Change of goal

Currently, the model does not perform very well. It struggles to differ low and high delay. We therefore decided to change the objective of the model. We are going to discretize the delay into bins and predict in which the delay will end. This way, we have a classification task. The reason for this change is that we believe it more relevant to have a model that can tell if the bus is 8 or 0 minutes delayed than if it is 43 or 52 seconds delayed. The classification is also easier to evaluate, and the model outputs an implicit probability distribution which we can use for the confidence.

In [ ]:
# function to discretize the delay variable into 10 bins
def discretize(x):
    # first, a bin for 0 or negative delay
    if x<=0:
        return 0
    # then a bin for more than 8 minutes delay
    elif x>8*60:
        return 9
    
    #finally, every delay is binned to 
    else:
        return x//60+1
    
# making udf of discretize
disc_udf = F.udf(lambda z: discretize(z))

In [ ]:
# discretizing the delay variable
vector_df = vector_df.withColumn('delay', disc_udf('delay').cast('int'))

We also tried sampling the dataset to account for class imbalances, but the approach was not too succesful

In [ ]:
import numpy as np
# calculating the number of samples in each bin
a = vector_df.groupBy('delay').count().sort('delay').select('count').collect()
minimum = (np.min(np.asarray([element[0] for element in a]))) # minimum in a bin

# creating a dictionary containg fractions to enable sampling to account for class imbalances
b = [float(minimum)/float(element[0]) for element in a]
fractions = {0:b[0], 1:b[1], 2:b[2], 3:b[3], 4:b[4], 5:b[5], 6:b[6], 7:b[7], 8:b[8], 9:b[9]}

# sampling the dataset to have as many of all classes
#vector_df = vector_df.sampleBy('delay', fractions = fractions)

In [ ]:
vector_df.groupBy('delay').count().show()

In [ ]:
# splitting in training and test data
splits = vector_df.randomSplit([0.8, 0.2])
train_df = splits[0]
test_df = splits[1]

### Modelling

Now, we implement a random forest classifier to solve the problem.

In [ ]:
from numpy import allclose
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import RandomForestClassifier, LogisticRegression, MultilayerPerceptronClassifier

# fitting the random forest classifier
rf = RandomForestClassifier(numTrees=1000, maxDepth=5, labelCol='delay', seed=42, maxMemoryInMB=64)
model = rf.fit(train_df)

In [ ]:
print('Importance of feature x in the final model:')
print('--------------------------------------------------')
feat = features +['transport type']
for i, element in enumerate(feat):
    print('Feature: %s, Importance: %.3f'%(element, model.featureImportances[i]))
    

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# creating the baseline, namely predicting one for each sample
@F.udf
def baseline(x):
    return 1

# transforming data
train_result = model.transform(train_df)
# adding baseline column
train_result = train_result.withColumn('baseline', baseline('delay').cast('double'))

# creating evaluator and calculating scores for predictions
evaluator = MulticlassClassificationEvaluator(labelCol="delay", predictionCol="prediction")
accuracy = evaluator.evaluate(train_result, {evaluator.metricName: "accuracy"})
f1 = evaluator.evaluate(train_result, {evaluator.metricName: "f1"})
precision = evaluator.evaluate(train_result, {evaluator.metricName: "weightedPrecision"})
recall = evaluator.evaluate(train_result, {evaluator.metricName: "weightedRecall"})

# creating evaluator and calculating scores for baseline
evaluator = MulticlassClassificationEvaluator(labelCol="delay", predictionCol="baseline")
b_accuracy = evaluator.evaluate(train_result, {evaluator.metricName: "accuracy"})
b_f1 = evaluator.evaluate(train_result, {evaluator.metricName: "f1"})

# baseline is calculated as the accuracy for guessing between 0 and 1 minute delay for every prediction
print("Accuracy = %.3f" % (accuracy))
print("Baseline Accuracy = %.3f" % (b_accuracy))
print("F1 = %.3f" % (f1))
print("Baseline F1 = %.3f" % (b_f1))
print("Precision = %.3f" % (precision))
print("Recall = %.3f" % (recall))


In [ ]:
# predicting on test set
result = model.transform(test_df)
result.select('delay', 'prediction').show(10)

In [ ]:
# inspecting if the model is good in the cases where there is more than 5 min delay
# likely, that this will be the most important examples in practice
result.filter(result.delay>5).select('delay', 'prediction').show()

In [ ]:
# further inspecting these examples by studying the probabilities
result.filter(result.delay>5).select('delay', 'probability').take(1)

In [ ]:
# calculating metrics on the test set
evaluator = MulticlassClassificationEvaluator(labelCol="delay", predictionCol="prediction")
accuracy = evaluator.evaluate(result, {evaluator.metricName: "accuracy"})
f1 = evaluator.evaluate(result, {evaluator.metricName: "f1"})
precision = evaluator.evaluate(result, {evaluator.metricName: "weightedPrecision"})
recall = evaluator.evaluate(result, {evaluator.metricName: "weightedRecall"})

print("Accuracy = %.3f" % (accuracy))
print("F1 = %.3f" % (f1))
print("Precision = %.3f" % (precision))
print("Recall = %.3f" % (recall))


#### Cross validation

In [ ]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# optimizing hyperparamters by cross validation
paramGrid = ParamGridBuilder() \
    .baseOn({rf.labelCol: 'delay'}) \
    .addGrid(rf.maxDepth, [4, 6, 8]) \
    .addGrid(rf.numTrees, [100, 500, 1000]) \
    .build()

crossval = CrossValidator(estimator=rf,
                          estimatorParamMaps=paramGrid,
                          evaluator=MulticlassClassificationEvaluator(),
                          numFolds=5)

cvModel = crossval.fit(train_df, {'labelCol':'delay'})

prediction = cvModel.transform(test)
evaluator = MulticlassClassificationEvaluator(labelCol="delay", predictionCol="prediction")
accuracy = evaluator.evaluate(prediction, {evaluator.metricName: "accuracy"})
f1 = evaluator.evaluate(prediction, {evaluator.metricName: "f1"})
precision = evaluator.evaluate(prediction, {evaluator.metricName: "weightedPrecision"})
recall = evaluator.evaluate(prediction, {evaluator.metricName: "weightedRecall"})


#### Testing the models probability scores

In [ ]:
# further inspecting these examples by studying the probabilities
num_samples = 10000
a = result.select('delay', 'probability').take(num_samples)

In [ ]:
count_fault = 0
for element in a:
    delay = element.delay
    cum = 0
    for i in range(delay):
        cum+=element.probability[i]
    
    if cum > 0.9:
        count_fault+=1

print(float(count_fault)/num_samples)

## Other models

We also tried implementing other models, but found the Random Forest to be the most promising and decided to down prioritize optimization of these algorithms. Please note that the feature column must be onehotencoded for lr and MLP to work properly.

#### Logistic Regression

In [ ]:
# accuracy: 0.426, F1 = 0.291
lr = LogisticRegression(maxIter=30, regParam=0, elasticNetParam=0, labelCol = label)
model = lr.fit(train_df)

#### Neural Network (MLP)

In [ ]:
# accuracy 0.431

# specify layers for the neural network:
# input layer of size 4 (features), two intermediate of size 5 and 4
# and output of size 3 (classes)
layers = [52, 128, 128, 12]

# create the trainer and set its parameters
#trainer = MultilayerPerceptronClassifier(maxIter=100, layers=layers, blockSize=128, seed=1234, labelCol=label)

# train the model
#model = trainer.fit(train_df)

# compute accuracy on the test set
#result = model.transform(test_df)
predictionAndLabels = result.select("prediction", 'delay')
evaluator = MulticlassClassificationEvaluator(metricName="accuracy", labelCol='delay')
print("Test set accuracy = " + str(evaluator.evaluate(predictionAndLabels)))

### Analysis of delays



As we decided to implement the routing algorithm locally, the inference from the predictive model must also be local. We decided to drop the ml predictive model of two reasons. 1) The model we developed was not much better than baseline. There was almost no correlation between our features and the delay. The only thing that correlated strongly was the departure at the last stop. However, we do not have access to this in real-time and can therefore not use it as a feature. If we were to make our work production-ready, we would accquire this data such that we could use real-time delays to predict future delays. 2) it was dificult to figure out how to move ml weights out of Spark and then run the model. Therefore, we rather decided to fit a probability distribution to the delays and use this as a measure of uncertainty.

More information about our approaches to predictive modelling can be found in predictive_modelling.ipynb, but we present the benfits and drawbacks in short here:

__Benfits__
- Easy and simple
- Work directly with a probability distribution which is what we try to model

__Drawbacks__
- We don't use the predictive power of the other features that we know, so it is not as accuracte as it could be

If we have more time, we would use more time creating a better predictive model. It could also be interesting to incoporate real-time data into the modelling as the strongest correlator of delay is previous delay, but this is a large scope.


In delays, there are two things we want to model: 1) How likely you are to miss a connections, and 2) How likely you are to not arrive at time at the last stop.

$T_D$ = Actual time of departure
$T_A$ = Actual time of arrival
$S_D$ = Scheduled time of departure
$S_A$ = Scheduled time of arrival
$A$ = Delay in arrival time
$D$ = Delay in departure time

We then want to find for 1):

$P({T_D-T_A \geq 0 })$
= $P({S_D + D - S_A + A \geq 0})$
= $P({A-D \leq S_D - S_A})$

By also accounting for transfer time are going to model:
$P({A-D \leq S_D - S_A-2})$

which corresponds to the CDF of the random variable $A-D$. We must therefore find this distribution.

In 2), we only need to model:

$P({A \leq B_T - S_A})$

where $B_T$ = the demanded arrival by time.



In [ ]:
# reading data from hdfs
sample = spark.read.orc('/group/gutane/processed_features')

In [ ]:
sample.printSchema()

In [ ]:
sample.count()

In [ ]:
#in the predictive modelling notebook we have seen that transport type is what correlates the most with delay
# and that train have different delays than bus and tram
# therefore, we split in train and other for the model fittings

# creating new column for departure delay
sample = sample.withColumn('departure_delay', F.col('actual_departure_time').cast('long')-F.col('scheduled_departure_time').cast('long'))
sample = sample.filter(sample.departure_delay.isNotNull())

#taking a sample of departure and arrival delays
delay_train = sample.filter(sample.transport_type=='Zug').select('delay', 'departure_delay', 'transport_type')
delay_other = sample.filter(sample.transport_type!='Zug').select('delay', 'departure_delay', 'transport_type')
delay_train = delay_train.sample(0.1, seed=41)
delay_other = delay_other.sample(0.1, seed=41)


In [ ]:
%%spark -o delay_train -t df -n 5000

In [ ]:
%%spark -o delay_other -t df -n 5000

First, we model the arrival delays

In [ ]:
%%local
import seaborn as sns
fig, ax = plt.subplots(2,1,figsize=(12,12))
sns.histplot(delay_train.delay, ax=ax[0])
sns.histplot(delay_other.delay, ax=ax[1])
ax[0].set_xlim(-500,1000)
ax[0].set_title("Delay distribution for trains")
ax[1].set_xlim(-500,1000)
ax[1].set_title("Delay distribution for other transport types")

We observe that delay follows something that looks like a right-skewed normal distribution. The right tail is longer than the left. We therefore, try estimating the delays with a skewed normal distribution. It does also look like a log-normal distribution, but since it takes on negative values, and there is no left bound, this is not the case.

In [ ]:
%%local
from scipy import stats
from scipy.stats import skewnorm

# fitting the skewed normal distribution
a_t, loc_t, scale_t = stats.skewnorm.fit(delay_train.delay)
a_o, loc_o, scale_o = stats.skewnorm.fit(delay_other.delay)

print("Estimated values")
print("a                  loc                scale")
print(a_t, loc_t, scale_t)
print(a_o, loc_o, scale_o)

In [ ]:
%%local
import numpy as np
fig, ax = plt.subplots(2,1,figsize=(12,12))
sns.histplot(delay_train.delay, ax=ax[0], stat='density', label = "empirical density")
sns.histplot(delay_other.delay, ax=ax[1], stat='density', label = "empirical density")

x1 = np.linspace(skewnorm.ppf(0.00001, a_t, loc=loc_t, scale=scale_t),
                skewnorm.ppf(0.99999, a_t, loc=loc_t, scale=scale_t), 100)
x2 = np.linspace(skewnorm.ppf(0.00001, a_o, loc=loc_o, scale=scale_o),
                skewnorm.ppf(0.99999, a_o, loc=loc_o, scale=scale_o), 100)

ax[0].plot(x1, skewnorm.pdf(x1, a_t, loc=loc_t, scale=scale_t),
       'r-', lw=5, alpha=0.6, label='fitted pdf')
ax[1].plot(x2, skewnorm.pdf(x2, a_o, loc=loc_o, scale=scale_o),
       'r-', lw=5, alpha=0.6, label='fitted pdf')

ax[0].set_xlim(-500,1000)
ax[1].set_xlim(-500,1000)

ax[0].set_title("Distribution of delay and the fitted skewed normal pdf for trains")
ax[1].set_title("Distribution of delay and the fitted skewed normal pdf for other transportation")
ax[0].set_xlabel("Delay")
ax[0].set_ylabel("Density")
ax[1].set_xlabel("Delay")
ax[1].set_ylabel("Density")
ax[0].legend()
ax[1].legend()

In [ ]:
%%local
import numpy as np

fig, ax = plt.subplots(2,1,figsize=(12,12))
sns.ecdfplot(delay_train.delay, ax=ax[0], lw=2, label = "ECDF")
sns.ecdfplot(delay_other.delay, ax=ax[1], lw=2, label = "ECDF")


x1 = np.linspace(skewnorm.ppf(0.00001, a_t, loc=loc_t, scale=scale_t),
                skewnorm.ppf(0.99999, a_t, loc=loc_t, scale=scale_t), 100)
x2 = np.linspace(skewnorm.ppf(0.00001, a_o, loc=loc_o, scale=scale_o),
                skewnorm.ppf(0.99999, a_o, loc=loc_o, scale=scale_o), 100)

ax[0].plot(x1, skewnorm.cdf(x1, a_t, loc=loc_t, scale=scale_t),
       'r-', lw=2, alpha=1, label='fitted CDF')
ax[1].plot(x2, skewnorm.cdf(x2, a_o, loc=loc_o, scale=scale_o),
       'r-', lw=2, alpha=1, label='fitted CDF')

ax[0].set_xlim(skewnorm.ppf(0.00001, a_t, loc=loc_t, scale=scale_t),
                skewnorm.ppf(0.99999, a_t, loc=loc_t, scale=scale_t))
ax[1].set_xlim(skewnorm.ppf(0.00001, a_t, loc=loc_t, scale=scale_t),
                skewnorm.ppf(0.99999, a_t, loc=loc_t, scale=scale_t))

ax[0].set_title("Empirical cumulative distribution of delay and the fitted skewed normal cdf for trains")
ax[1].set_title("Empirical cumulative distribution of delay and the fitted skewed normal cdf for other transportation")
ax[0].set_xlabel("Delay")
ax[0].set_ylabel("Density")
ax[1].set_xlabel("Delay")
ax[1].set_ylabel("Density")
ax[0].legend()
ax[1].legend()

We see that the distribution fit the data quite good. It does quite capture the spike around the mean, but otherwise good. Especially, it seems to capture the tails which may be the most important for our task as large delays are more capable of destroying a route plan. However, a weakness in our model is that we will underestimate the number of delays between 50 and 150 seconds for other transportation. Here, there is room for improvement.

In [ ]:
%%local
print("Probability that the delay is smaller than 60 seconds for trains: %.3f"%stats.skewnorm.cdf(60,a_t, loc_t, scale_t))
print("Probability that the delay is smaller than 60 seconds for other transport types: %.3f"%stats.skewnorm.cdf(60,a_o, loc_o, scale_o))


Then, we model the difference in arrival and departure delay.

In [ ]:
%%local
fig, ax = plt.subplots(2,1,figsize=(12,12))
sns.histplot(delay_train.delay-delay_train.departure_delay, ax=ax[0])
sns.histplot(delay_other.delay-delay_other.departure_delay, ax=ax[1])
ax[0].set_xlim(-200,200)
ax[0].set_title("Arrival minus departure delay distribution for trains")
ax[1].set_xlim(-200,200)
ax[1].set_title("Arrival minus departure delay distribution for other transport types")

The arrival minus distributions are harder to fit. The above one looks like some kind of Laplace distribution or similar. We tried several modelling approaches below, and found that the Laplace fittes best. For the other transport types, it is hard to find a fitting distributions, but we also here fit a Laplace distribution as we assume it should be similar as for trains.

In [ ]:
%%local
from scipy import stats
from scipy.stats import skewnorm, cauchy

# fitting the skewed normal distribution
a_t_stops, loc_t_stops, scale_t_stops = stats.skewnorm.fit(delay_train.delay-delay_train.departure_delay)
a_o_stops, loc_o_stops, scale_o_stops = stats.skewnorm.fit(delay_other.delay-delay_other.departure_delay)

print("Estimated values")
print("a                  loc                scale")
print(a_t, loc_t, scale_t)
print(a_o, loc_o, scale_o)

In [ ]:
%%local
from scipy import stats
from scipy.stats import skewnorm, cauchy, norm, laplace

# fitting the skewed normal distribution
loc_train_laplace, scale_train_laplace = stats.laplace.fit(delay_train.delay-delay_train.departure_delay)
loc_other_laplace, scale_other_laplace = stats.laplace.fit(delay_other.delay-delay_other.departure_delay)

fig, ax = plt.subplots(2,1,figsize=(12,12))
sns.histplot(delay_train.delay-delay_train.departure_delay, ax=ax[0], stat='density', label = "empirical density")
sns.histplot(delay_other.delay-delay_other.departure_delay, ax=ax[1], stat='density', label = "empirical density")

x1 = np.linspace(laplace.ppf(0.00001, loc_train_laplace, scale_train_laplace),
                laplace.ppf(0.99999,loc_train_laplace, scale_train_laplace), 100)
x2 = np.linspace(laplace.ppf(0.00001, loc_other_laplace, scale_other_laplace),
                laplace.ppf(0.99999,  loc_other_laplace, scale_other_laplace), 100)

ax[0].plot(x1, laplace.pdf(x1, loc_train_laplace, scale_train_laplace),
       'r-', lw=5, alpha=0.6, label='fitted pdf')
ax[1].plot(x2, laplace.pdf(x2,  loc_other_laplace, scale_other_laplace),
       'r-', lw=5, alpha=0.6, label='fitted pdf')

ax[0].set_xlim(-200,200)
ax[1].set_xlim(-200,200)

ax[0].set_title("Distribution of delay differences and the fitted laplace pdf for trains")
ax[1].set_title("Distribution of delay differences and the fitted laplace pdf for other transportation")
ax[0].set_xlabel("Delay")
ax[0].set_ylabel("Density")
ax[1].set_xlabel("Delay")
ax[1].set_ylabel("Density")
ax[0].legend()
ax[1].legend()

The Laplace distribution does an ok job of fitting the difference for trains, but not very much for other transportation. This is a weakness of our model, and something we would like to work more on, but we ran out of time and prioritize improving the routing algorithm. Further work should try developing a model that uses more of the available information to model the distributions.